In [1]:
# Configure the necessary Spark environment
import os
import sys

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

#os.environ['PYSPARK_SUBMIT_ARGS']="--jars /work/ericr/spark/sparkdev/postgresql.jar --executor-memory 40g --executor-cores 16 pyspark-shell"

# Initialize PySpark to predefine the SparkContext variable 'sc'
#execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

In [2]:
from pyspark.sql import SparkSession, SQLContext
import json


In [3]:
spark = SparkSession.builder.appName('Basic').config("spark.executor.extraJavaOptions","--executor-memory 40G --executor-cores 40 --driver-class-path $SPARK_HOME/postgresql.jar").getOrCreate()

In [4]:
dbconfig = {"url": "jdbc:postgresql://localhost/qxedb",
            "dbtable": "gateway_eventnotification",
            "user": "hermes",
            "password": "mysecret",
            "driver": "org.postgresql.Driver"}

In [ ]:


#import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /work/ericr/spark/sparkdev/postgresql.jar pyspark-shell'


In [5]:
df = spark.read.jdbc(url="jdbc:postgresql://localhost/qxedb", 
                      table="gateway_eventnotification",
                      properties=dbconfig)

In [6]:
sqlcontext = SQLContext(spark.sparkContext)

In [7]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- payload: string (nullable = true)
 |-- event_category_id: string (nullable = true)
 |-- table_data: string (nullable = true)



In [8]:
payload_df = sqlcontext.read.json(df.rdd.map(lambda r: r.payload))

In [9]:
payload_df.printSchema()

root
 |-- data_type: string (nullable = true)
 |-- events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dateTime: string (nullable = true)
 |    |    |-- eventCategory: string (nullable = true)
 |    |    |-- eventDetail: struct (nullable = true)
 |    |    |    |-- brand: string (nullable = true)
 |    |    |    |-- connectionInfo: struct (nullable = true)
 |    |    |    |    |-- localIpv4Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |    |-- sourceInSameSubnet: string (nullable = true)
 |    |    |    |    |-- sourceIpv4Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |    |-- sourceIpv6Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |   

In [10]:
versions = payload_df.select("version")

In [11]:
versions.show()

+-------+
|version|
+-------+
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|   null|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|   null|
|  1.1.0|
|  1.1.0|
|  1.1.0|
|  1.1.0|
+-------+
only showing top 20 rows

